In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSeq2SeqLM
import torch
from evaluate import load
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Tokenize BLEU hesaplama
def compute_token_bleu(prediction, reference):
    prediction_tokens = prediction.lower().split()
    reference_tokens = reference.lower().split()
    smooth = SmoothingFunction().method1  # Küçük veri için smoothing ekliyoruz.
    return sentence_bleu([reference_tokens], prediction_tokens, smoothing_function=smooth) * 100

# Metrik yükleme
rouge = load("rouge")

# F1 Skoru ve EM hesaplama için yardımcı fonksiyon
def compute_f1_em(prediction, reference):
    prediction_tokens = prediction.lower().split()
    reference_tokens = reference.lower().split()

    common_tokens = set(prediction_tokens) & set(reference_tokens)
    precision = len(common_tokens) / len(prediction_tokens) if prediction_tokens else 0
    recall = len(common_tokens) / len(reference_tokens) if reference_tokens else 0

    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0
    exact_match = 1.0 if prediction.strip().lower() == reference.strip().lower() else 0.0

    return f1, exact_match

# QA modeli için fonksiyon
def question_answering_model(tokenizer_path, model_path, question, context):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    model = AutoModelForQuestionAnswering.from_pretrained(model_path)
    model.to("cuda" if torch.cuda.is_available() else "cpu")

    inputs = tokenizer(question, context, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to("cuda")
    outputs = model(**inputs)

    start_index = outputs.start_logits.argmax(dim=-1).item()
    end_index = outputs.end_logits.argmax(dim=-1).item()
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index : end_index + 1]))
    return answer

# Seq2Seq modeli için fonksiyon
def seq2seq_model(tokenizer_path, model_path, question, context):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    model.to("cuda" if torch.cuda.is_available() else "cpu")

    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to("cuda")
    outputs = model.generate(inputs["input_ids"], max_length=64, num_beams=4)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Cosine Similarity hesaplama
def compute_cosine_similarity(prediction, reference):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([prediction.lower(), reference.lower()])
    cosine_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
    return cosine_sim

# Değerlendirme fonksiyonu
def evaluate_model(prediction, reference):
    prediction_lower = prediction.lower()
    reference_lower = reference.lower()

    rouge_result = rouge.compute(predictions=[prediction_lower], references=[reference_lower])["rouge1"]
    bleu_result = compute_token_bleu(prediction_lower, reference_lower)
    f1, em = compute_f1_em(prediction_lower, reference_lower)
    cosine_sim = compute_cosine_similarity(prediction_lower, reference_lower)

    return {"ROUGE": rouge_result, "BLEU": bleu_result, "F1": f1, "EM": em, "Cosine Similarity": cosine_sim}

# Çoklu context ve soru listesi
contexts_questions = [
    {
        "context": "Osman Bey 1258 yılında Söğüt’te doğdu. Osman Bey 1 Ağustos 1326’da Bursa’da hayatını kaybetmiştir. 1281 yılında Osman Bey 23 yaşında iken Ahi teşkilatından olan Şeyh Edebali’nin kızı Malhun Hatun ile evlendi. Bu evlilikten daha sonra Osmanlı Devleti’nin başına geçecek olan Orhan Gazi doğdu. Osmanlı Beyliği'nin ilk fethettiği ada İmralı Adasıdır. İmralı Adası 1308 yılında Osman Bey tarafından alınmıştır.",
        "qa_pairs": [
            {"question": "Osman Bey ne zaman doğmuştur?", "answer": "1258"},
            {"question": "Osman Bey nerede doğmuştur?", "answer": "Söğüt’te"},
            {"question": "İmralı Adası kim tarafından alınmıştır?", "answer": "Osman Bey"},
            {"question": "İmralı Adası hangi yılda alınmıştır?", "answer": "1308"},
            {"question": "Osman Bey’in eşinin adı nedir?", "answer": "Malhun Hatun"}
        ]
    },
    {
        "context": "Vasvar Antlaşması bir barış antlaşmasıdır. Vasvar Antlaşması 18 Ağustos 1618 tarihinde imzalanmıştır. Vasvar Antlaşması, Osmanlı Devleti ile Lehistan arasında imzalanmıştır. Vasvar Antlaşması Vasvar kentinde imzalanmıştır.",
        "qa_pairs": [
            {"question": "Vasvar Antlaşması ne çeşit bir antlaşmadır?", "answer": "barış antlaşması"},
            {"question": "Vasvar Antlaşması hangi tarihte imzalanmıştır?", "answer": "18 Ağustos 1618"},
            {"question": "Vasvar Antlaşması nerede imzalanmıştır?", "answer": "Vasvar kentinde"},
            {"question": "Vasvar Antlaşması Osmanlı Devleti ile hangi devlet arasında imzalanmıştır?", "answer": "Lehistan"},
            {"question": "Hangi antlaşma 18 Ağustos 1618 tarihinde imzalanmıştır?", "answer": "Vasvar Antlaşması"}
        ]
    },
    {
        "context": "Mostaganem Savaşı veya Mostaganem Muharebesi, 1558 yılında Osmanlı Devleti ile İspanyol İmparatorluğu arasında yapılan, Osmanlı zaferiyle sonuçlanan savaş. Osmanlı güçlerinin başında Hasan Paşa, İspanyol güçlerinin başında ise Kont Alkodete vardı.",
        "qa_pairs": [
            {"question": "Mostaganem Savaşı'nın diğer adı nedir?", "answer": "Mostaganem Muharebesi"},
            {"question": "Mostaganem Savaşı hangi tarihte oldu?", "answer": "1558 yılında"},
            {"question": "Mostaganem Savaşı hangi devletler arasında olmuştur?", "answer": "Osmanlı Devleti ile İspanyol İmparatorluğu"},
            {"question": "Osmanlı güçlerinin başında kim vardı?", "answer": "Hasan Paşa"},
            {"question": "İspanyol güçlerinin başında kim vardı?", "answer": "Kont Alkodete"}
        ]
    },
    {
        "context": "Celâlî isyanları, 16. ve 17. yüzyıllarda, Osmanlı yönetimindeki Anadolu'da Yavuz Sultan Selim döneminde başlayan ve IV. Mehmed dönemine kadar devam eden isyanlardır.",
        "qa_pairs": [
            {"question": "Celâlî isyanları hangi yüzyıllarda olmuştur?", "answer": "16. ve 17. yüzyıllarda"},
            {"question": "Celâlî isyanları hangi Osmanlı padişahı döneminde başlamıştır?", "answer": "Yavuz Sultan Selim"},
            {"question": "Celâlî isyanları hangi Osmanlı padişahı dönemine kadar devam etmiştir?", "answer": "IV. Mehmed"},
            {"question": "Celâlî isyanları hangi bölgede çıkmıştır?", "answer": "Anadolu'da"},
            {"question": "Celâlî tabiri kimle ilgilidir?", "answer": "Şeyh Celâl"}
        ]
    },
    {
        "context": "İstanbul'un Fethi, 6 Nisan-29 Mayıs 1453 tarihleri arasındaki kuşatmanın sonucunda Osmanlı Padişahı II. Mehmed komutasındaki birliklerin Bizans İmparatorluğu'nun başkenti İstanbul'u ele geçirmesiyle son bulmuştur.",
        "qa_pairs": [
            {"question": "İstanbul'un Fethi hangi tarihler arasında olmuştur?", "answer": "6 Nisan-29 Mayıs 1453"},
            {"question": "İstanbul'un Fethi hangi Osmanlı padişahı tarafından gerçekleştirilmiştir?", "answer": "II. Mehmed"},
            {"question": "İstanbul'un Fethi'nin sonucu ne olmuştur?", "answer": "Bizans İmparatorluğu'nun başkenti İstanbul ele geçirilmiştir"},
            {"question": "İstanbul daha önce kimler tarafından kuşatılmıştır?", "answer": "Emeviler ve Abbasiler"},
            {"question": "İstanbul'un Fethi'nin önemi nedir?", "answer": "Osmanlı'nın bir imparatorluk haline gelmesini sağlamıştır"}
        ]
    }
]


# Modellerin çalıştırılması ve değerlendirilmesi
results = {}

for context_data in contexts_questions:
    context = context_data["context"]
    qa_pairs = context_data["qa_pairs"]

    for qa in qa_pairs:
        question = qa["question"]
        expected_answer = qa["answer"]

        qa_answer_bert_english = question_answering_model("furkan_gunay/checkpoint-2500", "furkan_gunay/checkpoint-2500", question, context)
        seq2seq_answer_t5_english = seq2seq_model("abdulkerim_akan/custom-t5-turkish/checkpoint-8001", "abdulkerim_akan/custom-t5-turkish/checkpoint-8001", question, context)
        qa_answer_roberta_english = question_answering_model(
            "FacebookAI/roberta-base",
            "C:\\Users\\admin\\Desktop\\LLM_FINAL\\llm_final_project\\nisa_canbolat\\checkpoint-2500",
            question, context
        )
        qa_answer_bert_turkish = question_answering_model(
            "dbmdz/bert-base-turkish-cased",
            "C:\\Users\\admin\\Desktop\\LLM_FINAL\\llm_final_project\\turkce-bert\\checkpoint-1000",
            question, context
        )
        qa_answer_distilbert_turkish = question_answering_model(
            "dbmdz/distilbert-base-turkish-cased",
            "C:\\Users\\admin\\Desktop\\LLM_FINAL\\llm_final_project\\distilbert\\checkpoint-3500",
            question, context
        )
        qa_answer_turkce_electra = question_answering_model(
            "dbmdz/electra-base-turkish-cased-discriminator",
            "C:\\Users\\admin\\Desktop\\LLM_FINAL\\llm_final_project\\turkce-electra\\checkpoint-1500",
            question, context
        )

        results[f"{context}:{question}"] = {
            "Expected Answer": expected_answer,
            "BERT İngilizce Model Cevap": qa_answer_bert_english,
            "T5 İngilizce Model Cevap": seq2seq_answer_t5_english,
            "Roberta İngilizce Model Cevap": qa_answer_roberta_english,
            "BERT Türkçe Model Cevap": qa_answer_bert_turkish,
            "DistilBERT Türkçe Model Cevap": qa_answer_distilbert_turkish,
            "Electra Türkçe Model Cevap": qa_answer_turkce_electra,
            "BERT İngilizce Model Metrics": evaluate_model(qa_answer_bert_english, expected_answer),
            "T5 İngilizce Model Metrics": evaluate_model(seq2seq_answer_t5_english, expected_answer),
            "Roberta İngilizce Model Metrics": evaluate_model(qa_answer_roberta_english, expected_answer),
            "BERT Türkçe Model Metrics": evaluate_model(qa_answer_bert_turkish, expected_answer),
            "DistilBERT Türkçe Model Metrics": evaluate_model(qa_answer_distilbert_turkish, expected_answer),
            "Electra Türkçe Model Metrics": evaluate_model(qa_answer_turkce_electra, expected_answer),
        }

# Sonuçların yazdırılması
for question_context, metrics in results.items():
    print(f"\nQuestion: {question_context.split(':')[1]}")
    print(f"Beklenen Cevap: {metrics['Expected Answer']}")
    print(f"\nCevaplar:")
    print(f"  BERT İngilizce Model Cevap: {metrics['BERT İngilizce Model Cevap']}")
    print(f"  T5 İngilizce Model Cevap: {metrics['T5 İngilizce Model Cevap']}")
    print(f"  Roberta İngilizce Model Cevap: {metrics['Roberta İngilizce Model Cevap']}")
    print(f"  BERT Türkçe Model Cevap: {metrics['BERT Türkçe Model Cevap']}")
    print(f"  DistilBERT Türkçe Model Cevap: {metrics['DistilBERT Türkçe Model Cevap']}")
    print(f"  Electra Türkçe Model Cevap: {metrics['Electra Türkçe Model Cevap']}")
    print(f"\nDeğerlendirme Sonuçları:")
    for model, evaluation in metrics.items():
        if "Metrics" in model:
            print(f"  {model}:")
            for metric, value in evaluation.items():
                print(f"    {metric}: {value}")


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Question: Osman Bey ne zaman doğmuştur?
Beklenen Cevap: 1258

Cevaplar:
  BERT İngilizce Model Cevap: 1258
  T5 İngilizce Model Cevap: Söüt’te
  Roberta İngilizce Model Cevap:  1258
  BERT Türkçe Model Cevap: 1258
  DistilBERT Türkçe Model Cevap: 1258
  Electra Türkçe Model Cevap: 1258

Değerlendirme Sonuçları:
  BERT İngilizce Model Metrics:
    ROUGE: 1.0
    BLEU: 17.78279410038923
    F1: 1.0
    EM: 1.0
    Cosine Similarity: 1.0
  T5 İngilizce Model Metrics:
    ROUGE: 0.0
    BLEU: 0
    F1: 0.0
    EM: 0.0
    Cosine Similarity: 0.0
  Roberta İngilizce Model Metrics:
    ROUGE: 1.0
    BLEU: 17.78279410038923
    F1: 1.0
    EM: 1.0
    Cosine Similarity: 1.0
  BERT Türkçe Model Metrics:
    ROUGE: 1.0
    BLEU: 17.78279410038923
    F1: 1.0
    EM: 1.0
    Cosine Similarity: 1.0
  DistilBERT Türkçe Model Metrics:
    ROUGE: 1.0
    BLEU: 17.78279410038923
    F1: 1.0
    EM: 1.0
    Cosine Similarity: 1.0
  Electra Türkçe Model Metrics:
    ROUGE: 1.0
    BLEU: 17.78279410038

In [2]:
import csv
import pandas as pd

# Ortalamaları hesaplama
average_metrics = {
    "BERT İngilizce": {"F1": 0, "BLEU": 0, "EM": 0, "ROUGE": 0, "Cosine Similarity": 0},
    "T5 İngilizce": {"F1": 0, "BLEU": 0, "EM": 0, "ROUGE": 0, "Cosine Similarity": 0},
    "Roberta İngilizce": {"F1": 0, "BLEU": 0, "EM": 0, "ROUGE": 0, "Cosine Similarity": 0},
    "BERT Türkçe": {"F1": 0, "BLEU": 0, "EM": 0, "ROUGE": 0, "Cosine Similarity": 0},
    "DistilBERT Türkçe": {"F1": 0, "BLEU": 0, "EM": 0, "ROUGE": 0, "Cosine Similarity": 0},
    "Electra Türkçe": {"F1": 0, "BLEU": 0, "EM": 0, "ROUGE": 0, "Cosine Similarity": 0}
}

model_counts = {
    "BERT İngilizce": 0,
    "T5 İngilizce": 0,
    "Roberta İngilizce": 0,
    "BERT Türkçe": 0,
    "DistilBERT Türkçe": 0,
    "Electra Türkçe": 0
}

# Tüm sonuçlar üzerinden metrik ortalamalarını hesaplama
for metrics in results.values():
    for model in average_metrics.keys():
        model_metrics = metrics[f"{model} Model Metrics"]
        average_metrics[model]["F1"] += model_metrics["F1"]
        average_metrics[model]["BLEU"] += model_metrics["BLEU"]
        average_metrics[model]["EM"] += model_metrics["EM"]
        average_metrics[model]["ROUGE"] += model_metrics["ROUGE"]
        average_metrics[model]["Cosine Similarity"] += model_metrics["Cosine Similarity"]
        model_counts[model] += 1

# Ortalama değerleri hesaplama
for model, metrics in average_metrics.items():
    for metric in metrics.keys():
        metrics[metric] /= model_counts[model]

# CSV dosyasına yazma
output_file = "average_metrics.csv"
with open(output_file, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Model", "F1", "BLEU", "EM", "ROUGE", "Cosine Similarity"])

    for model, metrics in average_metrics.items():
        writer.writerow([model, metrics["F1"], metrics["BLEU"], metrics["EM"], metrics["ROUGE"], metrics["Cosine Similarity"]])

print(f"Ortalama metrikler '{output_file}' dosyasına kaydedildi.")


Ortalama metrikler 'average_metrics.csv' dosyasına kaydedildi.
